# National Park Service MCP Server Integration workbook

In [54]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [55]:
nps_api_server_params = [
    {"command": "uv", "args": ["run","nps.py"]}
]

# server1 = [
#     {"command": "uvx", "args": ["abc.py"]},
#     {"command": "uvx", "args": ["xyz.py"]}
# ]

In [56]:
nps_api_server = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in nps_api_server_params]

In [57]:
mcp_servers = nps_api_server
#mcp_servers = server1 + server2 +server3....

In [58]:
async def nps_agent(mcp_servers) -> Agent:
    instructions = """You are a National Park Service (NPS) Assistant, 
    an expert guide specializing in America's national parks, monuments, historic sites, and recreational areas. 
    Your mission is to help visitors discover, plan, and explore the incredible natural 
    and cultural treasures protected by the National Park Service.
    You use available tools to call NPS website API to provide accurate information
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [59]:


async def nps_tool(mcp_servers) -> Tool:
    researcher = await nps_agent(mcp_servers)
    return researcher.as_tool(
            tool_name="NPS-Researcher",
            tool_description="This tool researches online for park information, \
                either based on your specific request to look into a certain park, \
                Describe the park name to get further information"
        )



In [60]:
park_name = "North Cascades National Park"

for server in mcp_servers:
    print (server)
    await server.connect()
researcher = await nps_agent(mcp_servers) 
with trace("NPS-Researcher"):
    result = await Runner.run(researcher, park_name, max_turns=30)
display(Markdown(result.final_output))

North Cascades National Park is located in Washington state and offers an alpine landscape less than three hours from Seattle. It features jagged peaks crowned by more than 300 glaciers, forested valleys, and a diverse environment that responds to climate changes. Visitors can experience a variety of activities such as hiking, camping, biking, boating, rock climbing, and guided tours. The park is open year-round, with the main operational season from late May to late September.

Highlights:
- Alpine scenery with glaciers and mountains
- Forests, lakes, and diverse wildlife including wolves and wild cats
- Activities: hiking, camping (including backcountry and group camping), boating, biking, rock climbing, fishing, and more
- Visitor centers open late May to September
- Directions: Access via State Route 20, connecting to Interstate 5

For more information, you can visit the official website: https://www.nps.gov/noca/index.htm

Would you like information on specific activities, trails, camping, or other details?